# **multimoda-rs**: Tutorial examples IVUS/OCT to centerline
## Alining from files
The first example will focus on solving the problem of aligning frames within a pullback of either IVUS or OCT images.
We will start with gated IVUS images (systole/diastole) during two different states (e.g. rest/stress).
The .csv files are expected to set up in the following style:
```text
--------------------------------------------------------------------
|      185     |       5.32     |      2.37       |        0.0     |
|      185     |       5.12     |      2.46       |        0.0     |
|      ...     |       ...      |      ...        |        ...     |
```
where the first column is the frame index the point is from, the second to forth are x-, y- and z-coordinates. The naming conventions of the files are diastolic_contours.csv, diastolic_reference_points.csv, ... (see ./data). This is in alignment with the output of the [AIVUS-CAA software](https://https://github.com/AI-in-Cardiovascular-Medicine/AIVUS-CAA).

The first goal is to align the frames within a pullback by translating their centroids to a line and rotating them towards each other minimizing Hausdorff distance of the contours and catheter contours created from the image center. The influence of the catheter (which represents the image center) on the rotation can be adjusted by the number of points passed to catheter. If no catheter should be created just pass n_points=0.

In the same function states are aligned with each other (e.g. systole to diastole) and z-distance are averaged over the two states to have comparable frame positions. If heartrate is very different (e.g. rest to stress) a resampling is performed of the lower heartrate geometry.

Load packages multimodars, and for linking the numpy package.

In [1]:
import os
from pathlib import Path
import multimodars as mm
import numpy as np

# load the provided example data
os.chdir(Path.cwd().parent / "data")

# mode full compares diastole to systole for rest and stress conditions
rest, stress, dia, sys, _ = mm.from_file(
    mode="full", 
    rest_input_path="ivus_rest", 
    stress_input_path="ivus_stress", 
    steps_best_rotation=270, 
    range_rotation_deg=90, 
    rest_output_path="output/rest", 
    stress_output_path="output/stress", 
    diastole_output_path="output/diastole", 
    systole_output_path="output/systole", 
    interpolation_steps=28, 
    image_center=(4.5, 4.5),
    radius=0.5,
    n_points=20,
)

# preparing raw data to plot for comparison
rest_dia = np.genfromtxt("ivus_rest/diastolic_contours.csv")
rest_dia_ref_point = np.genfromtxt("ivus_rest/diastolic_reference_points.csv")
rest_sys= np.genfromtxt("ivus_rest/systolic_contours.csv")
rest_sys_ref_point = np.genfromtxt("ivus_rest/systolic_reference_points.csv")

rest_dia_geom_before = mm.numpy_to_geometry(
    contours_arr=rest_dia,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=rest_dia_ref_point,
)
new_contours = []
for contour in rest_dia_geom_before.contours:
    cont = contour.sort_contour_points()
    new_contours.append(cont)
rest_dia_geom_before.contours = np.array(new_contours)

rest_sys_geom_before = mm.numpy_to_geometry(
    contours_arr=rest_sys,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=rest_sys_ref_point,
)
new_contours = []
for contour in rest_sys_geom_before.contours:
    cont = contour.sort_contour_points()
    new_contours.append(cont)
rest_sys_geom_before.contours = np.array(new_contours)
print(rest_dia_geom_before)
mm.to_obj(rest_dia_geom_before, "output/unprocessed", walls=False, catheter=False, filename_contours="rest_dia.obj")
mm.to_obj(rest_sys_geom_before, "output/unprocessed", walls=False, catheter=False, filename_contours="rest_sys.obj")

Generating geometry for "ivus_stress"
file/path                                          loaded
ivus_stress/diastolic_contours.csv                 true
ivus_stress/diastolic_reference_points.csv         true
ivus_stress/combined_sorted_manual.csv             true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/diastolic_contours.csv                   true
ivus_rest/diastolic_reference_points.csv           true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/systolic_contours.csv                    true
ivus_rest/systolic_reference_points.csv            true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: systolic geometry generated
Generating geometry for "ivus_stress"
file/path                                  

In [2]:
# Install if needed
%pip install trimesh plotly

# Imports
import trimesh
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def trimesh_to_mesh3d(mesh, color, name):
    """
    Convert a trimesh.Trimesh to a Plotly Mesh3d trace.
    """
    # get vertices and faces
    verts = mesh.vertices
    faces = mesh.faces
    return go.Mesh3d(
        x=verts[:,0], y=verts[:,1], z=verts[:,2],
        i=faces[:,0], j=faces[:,1], k=faces[:,2],
        color=color, opacity=0.6,
        name=name,
        flatshading=True
    )

def plot_pair(before_paths, after_paths, colors, titles):
    """
    before_paths, after_paths: list of two .obj file paths [dia, sys]
    colors: list of two colors (e.g. ['blue','red'])
    titles: [left_title, right_title]
    """
    before_meshes = [trimesh.load(p) for p in before_paths]
    after_meshes  = [trimesh.load(p) for p in after_paths]

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type":"scene"}, {"type":"scene"}]],
        subplot_titles=titles
    )

    for mesh, color, label in zip(before_meshes, colors, ["diastole","systole"]):
        fig.add_trace(
            trimesh_to_mesh3d(mesh, color, f"before_{label}"),
            row=1, col=1
        )
    for mesh, color, label in zip(after_meshes, colors, ["diastole","systole"]):
        fig.add_trace(
            trimesh_to_mesh3d(mesh, color, f"after_{label}"),
            row=1, col=2
        )

    # link camera on both scenes
    camera = dict(
        eye=dict(x=1.5, y=1.5, z=1.0)
    )
    fig.update_layout(
        width=900, height=450,
        # apply same camera to both
        scene_camera=camera,
        scene2_camera=camera,
        # enforce equal scaling on x/y/z for both subplots
        scene=dict(
            aspectmode="data"
        ),
        scene2=dict(
            aspectmode="data"
        ),
        margin=dict(l=0, r=0, t=30, b=0)
    )
    fig.show()

# Paths “before” geometries
before_paths = [
    "output/unprocessed/rest_dia.obj",
    "output/unprocessed/rest_sys.obj",
]

# Paths “after” (processed) meshes
after_paths = [
    "output/rest/mesh_000_rest.obj",    # diastole post
    "output/rest/mesh_029_rest.obj",    # systole post
]

colors = ["royalblue", "firebrick"]

titles = ["Before Processing", "After Processing"]

plot_pair(before_paths, after_paths, colors, titles)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


The data is now neatly ordered in pairs (e.g. diastolic and systolic geometry). Every geometry has contours for lumen and walls and a created catheter. The reference point will be used to align the geometry to the centerline. All points corresponding to a contour are also save in a contour struct.

In [3]:
print(f"Example of PyGeometryPair:\n{rest}")
print(f"Example of PyGeometry:\n{rest.dia_geom}")
print(f"Example of PyContour:\n{rest.dia_geom.contours[0]}")
print(f"Example of PyContourPoint:\n{rest.dia_geom.contours[0].points[0]}")


Example of PyGeometryPair:
Diastolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false) 
Systolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=319, p=0, x=3.53, y=3.46, z=20.80, aortic=false)
Example of PyGeometry:
Geometry(17 contours, 17 walls), Catheter(17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false)
Example of PyContour:
Contour(id=0, points=501, centroid=(3.72, 5.25, 3.86))
Example of PyContourPoint:
Point(f=0, p=0, x=3.87, y=7.54, z=3.86, aortic=false)



The four pairs represent all 4 possible comparison in gated images, as for example in coronary artery anomalies (rest pulsatile lumen deformation, stress pulsatile lumen deformation, stress-induced diastolic lumen deformation and stress-induced systolic lumen deformation).

This can also be used for pre-and post-stenting comparison:

In [4]:
_, _, dia, sys, _ = mm.from_file(
    mode="full",
    rest_input_path="ivus_prestent",
    stress_input_path="ivus_poststent",
    rest_output_path="output/stent_rest",
    stress_output_path="output/stent_stress",
    diastole_output_path="output/stent_diastole",
    systole_output_path="output/stent_systole",
    steps_best_rotation=270,
    range_rotation_deg=90,
    interpolation_steps=0
    )

# cell – comparison of post-processing meshes
import trimesh
import plotly.graph_objects as go

# reuse the helper from before
def trimesh_to_mesh3d(mesh, color, name):
    verts = mesh.vertices
    faces = mesh.faces
    return go.Mesh3d(
        x=verts[:,0], y=verts[:,1], z=verts[:,2],
        i=faces[:,0], j=faces[:,1], k=faces[:,2],
        color=color, opacity=0.6,
        name=name,
        flatshading=True
    )

# load the two meshes
mesh_dia = trimesh.load("output/stent_diastole/mesh_000_diastolic.obj")
mesh_sys = trimesh.load("output/stent_diastole/mesh_001_diastolic.obj")

# create traces
trace_dia = trimesh_to_mesh3d(mesh_dia, 'royalblue', 'Before (mesh_000)')
trace_sys = trimesh_to_mesh3d(mesh_sys, 'firebrick', 'After (mesh_001)')

# define a canonical camera position
camera = dict(eye=dict(x=1.5, y=1.5, z=1.0))

# build and show figure
fig = go.Figure(data=[trace_dia, trace_sys])
fig.update_layout(
    title="Post-processing: Diastole vs Systole",
    width=600, height=600,
    scene=dict(
        aspectmode="data",    # equal scales on x/y/z
        camera=camera,
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    margin=dict(l=0, r=0, t=30, b=0)
)
fig.show()


Generating geometry for "ivus_prestent"
file/path                                          loaded
ivus_prestent/diastolic_contours.csv               true
ivus_prestent/diastolic_reference_points.csv       true
ivus_prestent/combined_sorted_manual.csv           true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_poststent"
file/path                                          loaded
ivus_poststent/diastolic_contours.csv              true
ivus_poststent/diastolic_reference_points.csv      true
ivus_poststent/combined_sorted_manual.csv          true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_prestent"
file/path                                          loaded
ivus_prestent/systolic_contours.csv                true
ivus_prestent/systolic_reference_points.csv        true
ivus_prestent/combined_sorted_manual.csv           true
geometry pair: systolic geometry generated
Generating geometry for "ivus_poststent"
file/path                    

However, for these pre- and poststenting comparisons the original from_file approach is computationally more expensive, than using the more flexible .from_array() approach:

In [ ]:
before_arr = np.genfromtxt("ivus_prestent/diastolic_contours.csv", delimiter='\t')
before_ref = np.genfromtxt("ivus_prestent/diastolic_reference_points.csv", delimiter='\t')
after_arr = np.genfromtxt("ivus_poststent/diastolic_contours.csv", delimiter='\t')
after_ref = np.genfromtxt("ivus_poststent/diastolic_reference_points.csv", delimiter='\t')

before_geom = mm.numpy_to_geometry(before_arr, np.array([]), np.array([]), before_ref)
after_geom = mm.numpy_to_geometry(after_arr, np.array([]), np.array([]), after_ref)

pair = mm.from_array(
    mode="singlepair",
    geometry_dia=before_geom,
    geometry_sys=after_geom,
    output_path="output/stent_comparison",
    steps_best_rotation=180,
    range_rotation_deg=60,
)

Or you can reconstruct a a single 3D geometry from OCT for example.

In [ ]:
oct_raw = np.genfromtxt("oct_single/oct_contours_raw.csv", delimiter=',')
oct_ref = np.genfromtxt("oct_single/oct_ref.csv", delimiter=',')
oct_geom = mm.numpy_to_geometry(
    contours_arr=oct_raw,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=oct_ref,
)

# this has an accuracy of 0.3° since 36 steps for +-6° range
oct_recon = mm.from_array(
    mode="single",
    geometry=oct_geom,
    steps_best_rotation=36,
    range_rotation_deg=6,
    image_center=(5.0, 5.0),
    radius=0.5,
    n_points=40,
    sort=False,
    interpolations_steps=0,
    write_obj=False,
)

Processing Geometry: None
Reference angle to vertical: -79.7 (°) 
 Rotating Reference by: 349.7 (°) 
 Added additional 180° rotation: true
+---------+------------+-------------------+----------------+-------+-------+-------------+
| Contour | Matched To | Relative Rot (°) | Total Rot (°) |  Tx   |  Ty   |  Centroid   |
+---------+------------+-------------------+----------------+-------+-------+-------------+
| 279     | 280        | -4.67             | 344.99         | -0.00 | -0.05 | (6.30,5.97) |
| 278     | 279        | 1.67              | 346.66         | -0.02 | -0.12 | (6.30,5.97) |
| 277     | 278        | 0.33              | 346.99         | 0.03  | -0.17 | (6.30,5.97) |
| 276     | 277        | 2.67              | 349.66         | 0.04  | -0.25 | (6.30,5.97) |
| 275     | 276        | -3.00             | 346.66         | 0.06  | -0.29 | (6.30,5.97) |
| 274     | 275        | 4.00              | 350.66         | 0.10  | -0.42 | (6.30,5.97) |
| 273     | 274        | 5.67      

## Alignment from array
While the alignment from file is one option, the more flexible option is to create Geometries directly from numpy array, and then perform the same operations with these Geometries. It is enough to provide contour coordinates and a reference point for the different states that should be compared.

In [3]:
dia_cont = np.genfromtxt("fixtures/idealized_geometry/diastolic_contours.csv", delimiter=',')
dia_ref = np.genfromtxt("fixtures/idealized_geometry/diastolic_reference_points.csv", delimiter=',')

sys_cont = np.genfromtxt("fixtures/idealized_geometry/systolic_contours.csv", delimiter=',')
sys_ref = np.genfromtxt("fixtures/idealized_geometry/systolic_reference_points.csv", delimiter=',')

rest_dia = mm.numpy_to_geometry(
    contours_arr=dia_cont,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=dia_ref,
)

rest_sys = mm.numpy_to_geometry(
    contours_arr=sys_cont,
    catheters_arr=np.array([]),
    walls_arr=np.array([]),
    reference_arr=sys_ref,
)

# Actual function call
rest, (dia_logs, sys_logs) = mm.from_array(
    mode="singlepair", 
    geometry_dia=rest_dia, 
    geometry_sys=rest_sys, 
    ouput_path="output/rest_array",
    steps_best_rotation=270,
    range_rotation_deg=90,
    interpolation_steps=0,
)
print(dia_logs)

Processing Geometry: Diastole
Reference angle to vertical: -89.4 (°) 
 Rotating Reference by: 359.4 (°) 
 Added additional 180° rotation: true
[(9, 10, -15.333333333333337, 344.0984455779998, 0.009999999999997122, -0.01000000000000334, 4.455950000000005, 4.218700000000003), (8, 9, -15.333333333333337, 328.76511224466645, 0.019999999999995133, -0.02000000000000579, 4.455950000000006, 4.218700000000004), (7, 8, -14.666666666666666, 314.0984455779998, 0.029999999999995808, -0.03000000000001002, 4.455950000000005, 4.218700000000004), (6, 7, -15.333333333333337, 298.7651122446665, 0.03999999999999915, -0.04000000000000892, 4.455950000000003, 4.218700000000005), (5, 6, -15.333333333333337, 283.43177891133314, 0.049999999999998046, -0.05000000000001137, 4.455950000000004, 4.218700000000006), (4, 5, -15.333333333333337, 268.09844557799977, 0.05999999999999872, -0.06000000000001382, 4.455950000000006, 4.2187000000000054), (3, 4, -14.666666666666666, 253.4317789113331, 0.06999999999999762, -0.07

## Reordering contours
Especially in IVUS images one problem is that breathing leads to bulk movements of frames relative to the catheter. In this case `multimoda-rs` offers the possibility to refine the ordering of the frames, by either providing records with the correct frame ordering or by using the function `refine_ordering`.